In [ ]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 28,
   "id": "732f6dac-d8fe-4e08-bcef-89ba07facd10",
   "metadata": {},
   "outputs": [],
   "source": [
    "import pandas as pd"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 29,
   "id": "b7122e7d-a82b-4f1e-8d17-eaa9e6dddf35",
   "metadata": {},
   "outputs": [],
   "source": [
    "# loading the data set of movie tile with its item_id\n",
    "anime_df = pd.read_csv('anime.csv')\n",
    "# print(anime_df.head())\n",
    "# print(\"Ratings Data set:\\n\")\n",
    "# loadint the data set of ratings of the user\n",
    "ratings_df = pd.read_csv('rating.csv')\n",
    "# print(ratings_df.head())"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 30,
   "id": "b942d770-5e27-4b8a-a7d4-f6eec61ea244",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "{'user_id': 1, 'anime_id': 24, 'rating': -1}\n",
      "{'anime_id': 32281, 'name': 'Kimi no Na wa.', 'genre': 'Drama, Romance, School, Supernatural', 'type': 'Movie', 'episodes': '1', 'rating': 9.37, 'members': 200630}\n"
     ]
    }
   ],
   "source": [
    "#converting the dataframe to dictionary\n",
    "user_anime_ratings = ratings_df.to_dict(orient = 'records')\n",
    "print(user_anime_ratings[1])\n",
    "#converting ratings dataframe to dictionary\n",
    "anime_names = anime_df.to_dict(orient = 'records')\n",
    "print(anime_names[0])\n",
    "# print(movie_names)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 31,
   "id": "54339ddc-d9a2-41b0-8710-c9f98fb623df",
   "metadata": {},
   "outputs": [],
   "source": [
    "def count_users():\n",
    "    #in this we are storing the record into set and in set there will be no duplicate\n",
    "    unique_user_id = set(record['user_id'] for record in user_anime_ratings)\n",
    "    return len(unique_user_id)\n",
    "\n",
    "#counting the number of movies which is nothing but the total values in movies names\n",
    "def count_animes():\n",
    "    unique_anime_id = set(record['anime_id'] for record in anime_names)\n",
    "    return len(unique_anime_id)\n",
    "\n",
    "def display_user_info(user_id):\n",
    "    user_info = [record for record in user_anime_ratings if record['user_id'] == user_id]\n",
    "    liked_threshold = 4\n",
    "    if not user_info:\n",
    "        print(f\"No information found for User ID: {user_id}\")\n",
    "        return  # Exit the function early if no user info found\n",
    "\n",
    "    print(f\"User ID: {user_id} - Anime Liked (Rating >= {liked_threshold}):\\n\")\n",
    "\n",
    "    for record in user_info:\n",
    "        anime_info = next((a for a in anime_names if a['anime_id'] == record['anime_id']), None)\n",
    "        if anime_info:\n",
    "            if record['rating'] >= liked_threshold:\n",
    "                print(f\"Anime: {anime_info['name']}, Rating: {record['rating']}\")\n",
    "            else:\n",
    "                print(f\"Anime ID: {record['anime_id']} was not liked (Rating: {record['rating']})\")\n",
    "        else:\n",
    "            print(f\"Anime ID: {record['anime_id']} not found in the anime list.\")\n",
    "\n",
    "def display_anime_name(anime_id):\n",
    "    anime_info = next((a for a in anime_names if a['anime_id'] == anime_id), None)\n",
    "    if not anime_info:\n",
    "        print(f\"No anime found with Anime ID: {anime_id}\")\n",
    "        return\n",
    "\n",
    "    satisfied_count = sum(1 for record in user_anime_ratings if record['anime_id'] == anime_id and record['rating'] >= 4)\n",
    "    unsatisfied_count = sum(1 for record in user_anime_ratings if record['anime_id'] == anime_id and record['rating'] < 4)\n",
    "\n",
    "    print(f\"Anime Name: {anime_info['name']}\")\n",
    "    print(f\"Satisfied Users (Rating >= 4): {satisfied_count}\")\n",
    "    print(f\"Unsatisfied Users (Rating < 4): {unsatisfied_count}\")\n",
    "\n",
    "\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 32,
   "id": "cbb19691-043d-4663-9b7f-8177a54f9215",
   "metadata": {},
   "outputs": [],
   "source": [
    "def recommend_anime(user_id):\n",
    "    # Get the target user's ratings\n",
    "    target_user_ratings = {record['anime_id']: record['rating'] for record in user_anime_ratings if record['user_id'] == user_id}\n",
    "\n",
    "    # Find anime that the target user likes\n",
    "    target_user_likes = {item for item, rating in target_user_ratings.items() if rating >= 4}\n",
    "\n",
    "    anime_recommendations = {}\n",
    "\n",
    "    # Iterate over each anime liked by the user to find similar animes\n",
    "    for liked_anime_id in target_user_likes:\n",
    "        liked_anime_info = next((anime for anime in anime_df.to_dict('records') if anime['anime_id'] == liked_anime_id), None)\n",
    "        \n",
    "        if not liked_anime_info or not isinstance(liked_anime_info['genre'], str):\n",
    "            continue  # Skip if anime info is not found or genre is not a string\n",
    "\n",
    "        liked_genres = set(liked_anime_info['genre'].split(','))  # Assuming genre is a comma-separated string\n",
    "\n",
    "        for anime in anime_df.to_dict('records'):\n",
    "            if anime['anime_id'] == liked_anime_id:\n",
    "                continue  # Skip the anime the user already liked\n",
    "\n",
    "            # Ensure genre is valid\n",
    "            if not isinstance(anime['genre'], str):\n",
    "                continue  # Skip if genre is not a string\n",
    "\n",
    "            # Calculate genre similarity\n",
    "            anime_genres = set(anime['genre'].split(','))\n",
    "            similarity = len(liked_genres.intersection(anime_genres))\n",
    "\n",
    "            # Add to recommendations based on similarity\n",
    "            if similarity > 0:\n",
    "                anime_recommendations[anime['anime_id']] = anime_recommendations.get(anime['anime_id'], 0) + similarity\n",
    "\n",
    "    # Sort recommendations by similarity\n",
    "    recommended_anime = sorted(anime_recommendations.items(), key=lambda x: x[1], reverse=True)\n",
    "    top_recommended_anime = recommended_anime[:5]\n",
    "\n",
    "    # Print recommended anime\n",
    "    print(f\"\\nRecommended Anime for User ID {user_id}:\")\n",
    "    if not top_recommended_anime:\n",
    "        print(\"No recommendations available.\")\n",
    "    else:\n",
    "        for anime_id, score in top_recommended_anime:\n",
    "            anime_info = next((a for a in anime_df.to_dict('records') if a['anime_id'] == anime_id), None)\n",
    "            if anime_info:\n",
    "                print(f\"Anime ID: {anime_id}, Title: {anime_info['name']} (Similarity Score: {score})\")\n",
    "\n",
    "\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 33,
   "id": "ca742bcf-04f4-410b-bd71-81a50aafa4b3",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "\n",
      "Menu:\n",
      "1. Count Users\n",
      "2. Count Animes\n",
      "3. Display User Information\n",
      "4. Display Anime Name\n",
      "5. Recommend Animes\n",
      "6. Exit\n"
     ]
    },
    {
     "name": "stdin",
     "output_type": "stream",
     "text": [
      "Enter your choice (1-6):  5\n",
      "Enter User ID:  1\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "\n",
      "Recommended Anime for User ID 1:\n",
      "Anime ID: 1840, Title: Zero no Tsukaima: Futatsuki no Kishi (Similarity Score: 16)\n",
      "Anime ID: 24703, Title: High School DxD BorN (Similarity Score: 16)\n",
      "Anime ID: 32215, Title: High School DxD BorN: Yomigaeranai Fushichou (Similarity Score: 16)\n",
      "Anime ID: 1195, Title: Zero no Tsukaima (Similarity Score: 16)\n",
      "Anime ID: 3712, Title: Zero no Tsukaima: Princesses no Rondo (Similarity Score: 16)\n",
      "\n",
      "Menu:\n",
      "1. Count Users\n",
      "2. Count Animes\n",
      "3. Display User Information\n",
      "4. Display Anime Name\n",
      "5. Recommend Animes\n",
      "6. Exit\n"
     ]
    },
    {
     "name": "stdin",
     "output_type": "stream",
     "text": [
      "Enter your choice (1-6):  5\n",
      "Enter User ID:  3\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "\n",
      "Recommended Anime for User ID 3:\n",
      "Anime ID: 231, Title: Asagiri no Miko (Similarity Score: 272)\n",
      "Anime ID: 6811, Title: InuYasha: Kanketsu-hen (Similarity Score: 266)\n",
      "Anime ID: 4938, Title: Tsubasa: Shunraiki (Similarity Score: 266)\n",
      "Anime ID: 249, Title: InuYasha (Similarity Score: 266)\n",
      "Anime ID: 121, Title: Fullmetal Alchemist (Similarity Score: 257)\n",
      "\n",
      "Menu:\n",
      "1. Count Users\n",
      "2. Count Animes\n",
      "3. Display User Information\n",
      "4. Display Anime Name\n",
      "5. Recommend Animes\n",
      "6. Exit\n"
     ]
    },
    {
     "name": "stdin",
     "output_type": "stream",
     "text": [
      "Enter your choice (1-6):  6\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Exiting program.\n"
     ]
    }
   ],
   "source": [
    "# Menu-driven program\n",
    "def menu():\n",
    "    while True:\n",
    "        print(\"\\nMenu:\")\n",
    "        print(\"1. Count Users\")\n",
    "        print(\"2. Count Animes\")\n",
    "        print(\"3. Display User Information\")\n",
    "        print(\"4. Display Anime Name\")\n",
    "        print(\"5. Recommend Animes\")\n",
    "        print(\"6. Exit\")\n",
    "        \n",
    "        choice = int(input(\"Enter your choice (1-6): \"))\n",
    "        \n",
    "        if choice == 1:\n",
    "            print(f\"Total Users: {count_users()}\")\n",
    "        elif choice == 2:\n",
    "            print(f\"Total Animes: {count_animes()}\")\n",
    "        elif choice == 3:\n",
    "            user_id = int(input(\"Enter User ID: \"))\n",
    "            display_user_info(user_id)\n",
    "        elif choice == 4:\n",
    "            anime_id = int(input(\"Enter Anime ID: \"))\n",
    "            display_anime_name(anime_id)\n",
    "            \n",
    "        elif choice == 5:\n",
    "            user_id = int(input(\"Enter User ID: \"))\n",
    "            recommendations = recommend_anime(user_id)\n",
    "        elif choice == 6:\n",
    "            print(\"Exiting program.\")\n",
    "            break\n",
    "        else:\n",
    "            print(\"Invalid choice. Please try again.\")\n",
    "\n",
    "# Start the menu-driven program\n",
    "menu()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "dbfb500e-877d-4c7e-aa0b-e092287cf028",
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.12.4"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}
